In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
#from progressbar import *  
from tqdm._tqdm import trange
import warnings
import gc
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('display.max_columns',None)

In [3]:
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])

app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])

In [4]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    # 统计出共有多少种类别的app
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')    

            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']


            df = df.merge(app_info, how='left', on='appId')
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
            #print(df.groupby(['uid'])['category'].value_counts().index[0])
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [5]:
processUserAppUsage()

index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 1

In [5]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [6]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
# 将之前训练的appuseProcess.csv进行合并，只统计激活的app的个数

# 进阶——对user_app_actived.csv高级统计——统计用户安装的每种APP的数目

def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)
    #print(resTable[:5])
    return resTable

import csv

def SaveFile2csv(path,contents):
    f=open(path,'a+')
    writer=csv.writer(f)
    writer.writerow(contents)
    f.close()

In [8]:
#  cats = list(set(app_info['category']))
#  category2id = dict(zip(sorted(cats), range(0,len(cats))))
#  id2category = dict(zip(range(0,len(cats)), sorted(cats)))
#  app_info['category'] = app_info['category'].apply(lambda x: category2id[x])


# 统计出共有多少种类别的app
#app_info = pd.read_csv("app_info.csv", names=['appId','category'])
#cats = list(set(app_info['category']))
#category2id = dict(zip(sorted(cats), range(0,len(cats))))
#id2category = dict(zip(range(0,len(cats)), sorted(cats)))
#app_info['category'] = app_info['category'].apply(lambda x: category2id[x])



#print(app_info)
#print(len(set(app_info['category'])))
#print(id2category)
#print(category2id)

In [28]:
temp = mergeBasicTables(age_train)
temp.head()

,uid,age_group,gender,city,prodName,ramCapacity,ramLeftRation,romCapacity,romLeftRation,color,fontSize,ct,carrier,os,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum
0,1000001,4,0,51,78,3.0,0.43,32.0,0.46,80,1.15,5,1,14,108,0.0,0.0,1.00,0.07,0.0,0.0,3319
1,1000011,3,0,30,138,NaN,NaN,NaN,NaN,16,NaN,6,1,15,0,0.0,0.0,0.00,0.00,0.0,0.0,220
2,1000015,5,1,228,78,3.0,0.34,32.0,0.06,80,1.30,8,2,14,12,0.0,0.0,0.03,0.13,0.0,0.0,21881
3,1000019,3,0,57,166,2.0,NaN,17.0,NaN,100,NaN,7,3,15,0,0.0,0.0,0.00,0.00,0.0,0.0,0
4,1000023,2,1,293,164,2.0,0.34,16.0,0.06,119,1.00,8,2,12,5,0.0,0.0,0.00,0.13,0.0,0.0,0


In [3]:
appuseProcessed_df = pd.read_csv("appuseProcessed.csv")
appuseProcessed_df.head()

,uid,usage_cnt,usage_appid_cnt,usage_date_cnt,duration_mean,duration_max,duration_std,times_mean,times_max,times_std,category,usage_most_used_category
0,1000364,198,13,30,1973.515152,44845.0,4600.297805,32.272727,241.0,48.347162,8,18.0
1,1000387,491,41,30,1986.160896,24954.0,3338.793126,63.010183,1442.0,169.986128,14,18.0
2,1000872,406,29,30,861.123153,11875.0,1912.555912,56.347291,604.0,119.863091,12,18.0
3,1000951,223,21,30,1677.932735,17073.0,2826.470570,28.520179,208.0,44.027717,11,18.0
4,1001091,1019,160,30,871.041217,33739.0,2445.063938,35.236506,1552.0,111.937386,20,18.0


In [33]:

aaa = pd.merge(temp,user_app_actived, how='left', on='uid')

user_app_actived.head()
#print(appusedTable)

,uid,appId
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...


In [32]:
aaa.head()

,uid,age_group,gender,city,prodName,ramCapacity,ramLeftRation,romCapacity,romLeftRation,color,fontSize,ct,carrier,os,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,appId
0,1000001,4,0,51,78,3.0,0.43,32.0,0.46,80,1.15,5,1,14,108,0.0,0.0,1.00,0.07,0.0,0.0,3319,a00140327#a00170298#a00184278#a00187480#a00239...
1,1000011,3,0,30,138,NaN,NaN,NaN,NaN,16,NaN,6,1,15,0,0.0,0.0,0.00,0.00,0.0,0.0,220,a00158535#a00163116#a00170432#a00187480#a00224...
2,1000015,5,1,228,78,3.0,0.34,32.0,0.06,80,1.30,8,2,14,12,0.0,0.0,0.03,0.13,0.0,0.0,21881,a00109386#a00170432#a0021880#a00244790#a002475...
3,1000019,3,0,57,166,2.0,NaN,17.0,NaN,100,NaN,7,3,15,0,0.0,0.0,0.00,0.00,0.0,0.0,0,a00157060#a00274701
4,1000023,2,1,293,164,2.0,0.34,16.0,0.06,119,1.00,8,2,12,5,0.0,0.0,0.00,0.13,0.0,0.0,0,a00170298#a00203358#a00275200#a00278905#a00289...


In [7]:
#tempBaseTable = mergeBasicTables(age_train)
#app_actived_table = tempBaseTable.merge(user_app_actived, how='left', on='uid')

#print(user_app_actived)
#def calculate_used_app_info_for_every_user(now_df, user_app_actived, app_info_dict):
#    for index, row in user_app_actived.iterrows():
#        appIdList = row["appId"].split('#')
        #print(appIdList)
        #print(index)
#        for appId in appIdList:
#            if appId in app_info_dict.keys():
#                now_df.at[index, str(app_info_dict[appId][0])] += 1
                #print(app_info_dict[appId][0])


## 第一次特征工程——结合app_info.csv统计用户激活的每类APP的个数

In [8]:
def mergeAppDataUpgrade(baseTable):
    
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))

    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)


    now_df = pd.DataFrame()
    now_df['uid'] = resTable['uid']
    for i in range(40):
        now_df[str(i)] = 0

    now_df = pd.merge(resTable, now_df, how='left', on='uid')

    now_df = now_df.fillna(0)

    #    now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
        #print(df.groupby(['uid'])['category'].value_counts().index[0])

        # 开始计算每类APP的个数
    #resTable = pd.merge(resTable, now_df, how='left', on='uid')



    # 注意给的数据中，用户激活的app中有一些是不在app_info 中的，可以先忽略或者单独归为一类
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    # 统计出共有多少种类别的app
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    
    # app_info转dict
    app_info_dict = app_info.set_index('appId').T.to_dict('list')

    now_df = pd.merge(now_df, user_app_actived, how='left', on='uid')
    
    try:
        with tqdm(range(0, len(now_df))) as t:
            for index in t:
                appIdList = now_df.at[index, 'appId_y'].split('#')
                for appId in appIdList:
                    if appId in app_info_dict.keys():
                        now_df.at[index, str(app_info_dict[appId][0])] += 1
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()     

    now_df = now_df.drop(['appId_y'], axis=1)
   
    return now_df

In [ ]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppDataUpgrade(mergeBasicTables(age_train))
df_test = mergeAppDataUpgrade(mergeBasicTables(age_test))
print(df_train.shape)
print(df_test.shape)

In [67]:
df_train.to_csv("add_40_actived_classes_features.csv",index=0)

In [7]:
# 第二次特征工程——使用user_app_usage，计算每类app的使用时长

df_train = pd.read_csv("add_40_actived_classes_features.csv")
#df_test = mergeAppDataUpgrade(mergeBasicTables(age_test))
df_train.head()


,uid,age_group,gender,city,prodName,ramCapacity,ramLeftRation,romCapacity,romLeftRation,color,fontSize,ct,carrier,os,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,FFuncSum,appId_x,usage_cnt,usage_appid_cnt,usage_date_cnt,duration_mean,duration_max,duration_std,times_mean,times_max,times_std,category,usage_most_used_category,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,1000001,4,0,51,78,3.0,0.43,32.0,0.46,80,1.15,5,1,14,108,0.0,0.0,1.00,0.07,0.0,0.0,3319,21,251.0,24.0,30.0,806.215139,12646.0,1651.473740,49.330677,396.0,72.519668,11.0,17.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,9,2,1,0,0,0,0,0,0,3,0,0,2,0,0,0,0,0,0,0,0,0,1
1,1000011,3,0,30,138,0.0,0.00,0.0,0.00,16,0.00,6,1,15,0,0.0,0.0,0.00,0.00,0.0,0.0,220,16,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,41.0,41.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,3,0,0,2,0,0,0,0,0,0,0,3,0,1,0,0,0,1,0,3,0,0
2,1000015,5,1,228,78,3.0,0.34,32.0,0.06,80,1.30,8,2,14,12,0.0,0.0,0.03,0.13,0.0,0.0,21881,19,87.0,10.0,19.0,2860.620690,14839.0,3242.297894,146.333333,991.0,231.758328,10.0,17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,2
3,1000019,3,0,57,166,2.0,0.00,17.0,0.00,100,0.00,7,3,15,0,0.0,0.0,0.00,0.00,0.0,0.0,0,2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,41.0,41.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1000023,2,1,293,164,2.0,0.34,16.0,0.06,119,1.00,8,2,12,5,0.0,0.0,0.00,0.13,0.0,0.0,0,13,112.0,8.0,29.0,2667.589286,70806.0,9595.549063,10.955357,87.0,15.857694,7.0,17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,2,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1,0,0


In [14]:
print(df_train.head())
print(df_test.head())

       uid  age_group  gender  city  prodName  ramCapacity  ramLeftRation  \
0  1000001          4       0    51        78          3.0           0.43   
1  1000011          3       0    30       138          0.0           0.00   
2  1000015          5       1   228        78          3.0           0.34   
3  1000019          3       0    57       166          2.0           0.00   
4  1000023          2       1   293       164          2.0           0.34   

   romCapacity  romLeftRation  color  fontSize  ct  carrier  os  bootTimes  \
0         32.0           0.46     80      1.15   5        1  14        108   
1          0.0           0.00     16      0.00   6        1  15          0   
2         32.0           0.06     80      1.30   8        2  14         12   
3         17.0           0.00    100      0.00   7        3  15          0   
4         16.0           0.06    119      1.00   8        2  12          5   

   AFuncTimes  BFuncTimes  CFuncTimes  DFuncTimes  EFuncTimes  FFunc

In [8]:

#df_train = pd.read_csv("add_40_actived_classes_features.csv")


## 第二次特征工程——迭代处理数据量较大的user_app_usage.csv，结合app_info.csv统计得到用户每一类APP的使用时长（duration）

In [8]:
def processUserAppUsageUpgrade(table, name):
    #if name == 'trainSet':
    #    resTable = pd.read_csv("add_40_actived_classes_features.csv")
    #else :
    #    resTable = table
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    i = 1
    while True:
        try:
            
            
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            # 最后的user去掉
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]
            
            # 添加列，计算每类app的使用时长
            for ii in range(40):
                df['duration' + str(ii)] = 0
            
            
            # 注意给的数据中，用户激活的app中有一些是不在app_info 中的，可以先忽略或者单独归为一类
            app_info = pd.read_csv("app_info.csv", names=['appId','category'])
            # 统计出共有多少种类别的app
            cats = list(set(app_info['category']))
            category2id = dict(zip(sorted(cats), range(0,len(cats))))
            id2category = dict(zip(range(0,len(cats)), sorted(cats)))
            app_info['category'] = app_info['category'].apply(lambda x: category2id[x])

            # app_info转dict
            app_info_dict = app_info.set_index('appId').T.to_dict('list')
            
            df_dict = df.to_dict(orient= 'dict')
            
            pbar = tqdm(total=len(df_dict['uid']))  
            first_user = 'abc'
            first_user_idx = 0
            
            for index in df_dict['uid']:
                pbar.update(1)
                
                appId = df_dict['appId'][index]
                curUID = df_dict['uid'][index]
                curDuration = df_dict['duration'][index]
                if curUID != first_user :
                    first_user = curUID
                    first_user_idx = index
                    
                
                if appId in app_info_dict.keys():
                    df_dict["duration" + str(app_info_dict[appId][0])][first_user_idx] += curDuration
                        #if index < 10:
                         #   print(df.head())
                          #  print(df.shape)
                           # print("first_user_idx:" + str(first_user_idx))
            pbar.close()
            
            #字典完再转回dataframe
            df = pd.DataFrame.from_dict(df_dict)
            #去除uid的重复项
            df = df.drop_duplicates(['uid'])
            df = df.drop(['appId', 'duration', 'times', 'use_date'], axis = 1)
            
            if i==2 :
                df.to_csv('process_user_app_usage.csv', mode='a', header=True)
                #resTable = resTable.fillna(0)
                #print(resTable.head())
                #for index, row in df.iterrows():
                #    if(row['uid'] == 1000364):
                #        print(row)
                #        break
            #break
            else:
                df.to_csv('process_user_app_usage.csv', mode='a', header=False)

            #print(df.head())
            
            # resTable = resTable.merge(df, how='left', on='uid')
            del df
            del df_dict
            gc.collect()
        except StopIteration:
            break
        
        
    #resTable.to_csv("2nd_featureEngineering_"+ name + ".csv",index=0)
    
    print("Iterator is stopped")
    #return resTable

In [9]:
processUserAppUsageUpgrade("", 'trainSet')


index: 1


100%|██████████████████████████████████████████████████████████████████████| 999900/999900 [00:01<00:00, 905662.05it/s]


index: 2


100%|██████████████████████████████████████████████████████████████████████| 999691/999691 [00:01<00:00, 914560.81it/s]


index: 3


...

index: 651


100%|████████████████████████████████████████████████████████████████████| 1000267/1000267 [00:01<00:00, 792962.58it/s]


index: 652


100%|██████████████████████████████████████████████████████████████████████████| 7743/7743 [00:00<00:00, 862566.62it/s]


index: 653
Iterator is stopped


In [9]:
tempdf = pd.read_csv('process_user_app_usage.csv')
tempdf.head()



,Unnamed: 0,uid,duration0,duration1,duration2,duration3,duration4,duration5,duration6,duration7,duration8,duration9,duration10,duration11,duration12,duration13,duration14,duration15,duration16,duration17,duration18,duration19,duration20,duration21,duration22,duration23,duration24,duration25,duration26,duration27,duration28,duration29,duration30,duration31,duration32,duration33,duration34,duration35,duration36,duration37,duration38,duration39
0,0,1000364,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,40589.0,0.0,32.0,30228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37765.0,0.0,392.0
1,198,1000387,0.0,0,0.0,0,0.0,0.0,0.0,6330.0,0.0,12871.0,0.0,109901.0,0.0,0.0,0.0,0.0,0.0,39409.0,195735.0,274784.0,2408.0,55456.0,0.0,0.0,0.0,0.0,1157.0,0.0,0.0,133202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50881.0,29069.0,6277.0
2,689,1000872,0.0,0,0.0,0,0.0,0.0,0.0,25421.0,0.0,17558.0,0.0,0.0,0.0,0.0,20772.0,0.0,0.0,5135.0,10821.0,5278.0,707.0,1683.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213779.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33108.0,320.0,14950.0
3,1095,1000951,0.0,0,0.0,0,0.0,0.0,0.0,724.0,0.0,13677.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10325.0,81252.0,678.0,0.0,0.0,0.0,0.0,188948.0,0.0,0.0,0.0,0.0,72441.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5243.0,0.0,787.0
4,1318,1001091,1271.0,0,0.0,0,0.0,0.0,0.0,2550.0,0.0,16552.0,0.0,23718.0,0.0,0.0,7326.0,0.0,0.0,51220.0,214100.0,11368.0,601.0,3221.0,8543.0,0.0,0.0,0.0,0.0,0.0,0.0,282536.0,0.0,7390.0,5854.0,6315.0,0.0,0.0,0.0,54666.0,16.0,13487.0


In [35]:
tempdf.drop(["Unnamed: 0"], axis = 1)


,uid,duration0,duration1,duration2,duration3,duration4,duration5,duration6,duration7,duration8,duration9,duration10,duration11,duration12,duration13,duration14,duration15,duration16,duration17,duration18,duration19,duration20,duration21,duration22,duration23,duration24,duration25,duration26,duration27,duration28,duration29,duration30,duration31,duration32,duration33,duration34,duration35,duration36,duration37,duration38,duration39
0,1000364,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,40589.0,0.0,32.0,30228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37765.0,0.0,392.0
1,1000387,0.0,0,0.0,0,0.0,0.0,0.0,6330.0,0.0,12871.0,0.0,109901.0,0.0,0.0,0.0,0.0,0.0,39409.0,195735.0,274784.0,2408.0,55456.0,0.0,0.0,0.0,0.0,1157.0,0.0,0.0,133202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50881.0,29069.0,6277.0
2,1000872,0.0,0,0.0,0,0.0,0.0,0.0,25421.0,0.0,17558.0,0.0,0.0,0.0,0.0,20772.0,0.0,0.0,5135.0,10821.0,5278.0,707.0,1683.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213779.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33108.0,320.0,14950.0
3,1000951,0.0,0,0.0,0,0.0,0.0,0.0,724.0,0.0,13677.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10325.0,81252.0,678.0,0.0,0.0,0.0,0.0,188948.0,0.0,0.0,0.0,0.0,72441.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5243.0,0.0,787.0
4,1001091,1271.0,0,0.0,0,0.0,0.0,0.0,2550.0,0.0,16552.0,0.0,23718.0,0.0,0.0,7326.0,0.0,0.0,51220.0,214100.0,11368.0,601.0,3221.0,8543.0,0.0,0.0,0.0,0.0,0.0,0.0,282536.0,0.0,7390.0,5854.0,6315.0,0.0,0.0,0.0,54666.0,16.0,13487.0
5,1001365,0.0,0,0.0,0,0.0,0.0,0.0,5472.0,0.0,4250.0,0.0,0.0,0.0,0.0,1581.0,0.0,0.0,4193.0,7825.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,87414.0,0.0,0.0,0.0,75.0,0.0,0.0,0.0,13788.0,0.0,27495.0
6,1001441,0.0,0,0.0,0,0.0,0.0,0.0,151790.0,0.0,1348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6192.0,195634.0,78716.0,1150.0,139516.0,712.0,0.0,0.0,0.0,0.0,0.0,0.0,160048.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45521.0,0.0,11675.0
7,1001627,0.0,0,0.0,0,0.0,0.0,0.0,3449.0,0.0,5184.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5711.0,81032.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129590.0,0.0,366847.0,0.0,0.0,0.0,0.0,0.0,25004.0,0.0,3931.0
8,1001700,0.0,0,0.0,0,44061.0,0.0,0.0,2753.0,0.0,14119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52482.0,56869.0,33709.0,110355.0,23291.0,0.0,0.0,27732.0,0.0,2624.0,0.0,0.0,127161.0,0.0,0.0,0.0,3831.0,0.0,0.0,0.0,10128.0,10997.0,2683.0
9,1001710,0.0,0,0.0,0,27833.0,0.0,0.0,33252.0,0.0,6896.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13561.0,43192.0,0.0,1876.0,160697.0,4429.0,0.0,0.0,0.0,372.0,0.0,0.0,155099.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29133.0,0.0,9638.0


In [37]:
tempdf.drop(["Unnamed: 0"], axis = 1).to_csv("process_user_app_usage_del_unname_column.csv",index=0)

In [10]:
tempdf = pd.read_csv('process_user_app_usage_del_unname_column.csv')
print(tempdf.head())
print(tempdf.shape)

       uid  duration0  duration1  duration2  duration3  duration4  duration5  \
0  1000364        0.0          0        0.0          0        0.0        0.0   
1  1000387        0.0          0        0.0          0        0.0        0.0   
2  1000872        0.0          0        0.0          0        0.0        0.0   
3  1000951        0.0          0        0.0          0        0.0        0.0   
4  1001091     1271.0          0        0.0          0        0.0        0.0   

   duration6  duration7  duration8  duration9  duration10  duration11  \
0        0.0        0.0        0.0        0.0         0.0         0.0   
1        0.0     6330.0        0.0    12871.0         0.0    109901.0   
2        0.0    25421.0        0.0    17558.0         0.0         0.0   
3        0.0      724.0        0.0    13677.0         0.0         0.0   
4        0.0     2550.0        0.0    16552.0         0.0     23718.0   

   duration12  duration13  duration14  duration15  duration16  duration17  \
0  

In [47]:
final_df = pd.read_csv("add_40_actived_classes_features.csv")
print(final_df.head())
print(final_df.shape)



       uid  age_group  gender  city  prodName  ramCapacity  ramLeftRation  \
0  1000001          4       0    51        78          3.0           0.43   
1  1000011          3       0    30       138          0.0           0.00   
2  1000015          5       1   228        78          3.0           0.34   
3  1000019          3       0    57       166          2.0           0.00   
4  1000023          2       1   293       164          2.0           0.34   

   romCapacity  romLeftRation  color  fontSize  ct  carrier  os  bootTimes  \
0         32.0           0.46     80      1.15   5        1  14        108   
1          0.0           0.00     16      0.00   6        1  15          0   
2         32.0           0.06     80      1.30   8        2  14         12   
3         17.0           0.00    100      0.00   7        3  15          0   
4         16.0           0.06    119      1.00   8        2  12          5   

   AFuncTimes  BFuncTimes  CFuncTimes  DFuncTimes  EFuncTimes  FFunc

In [48]:
final_df = pd.merge(final_df, tempdf, how='left', on='uid').fillna(0)
print(final_df.head())
print(final_df.shape)

       uid  age_group  gender  city  prodName  ramCapacity  ramLeftRation  \
0  1000001          4       0    51        78          3.0           0.43   
1  1000011          3       0    30       138          0.0           0.00   
2  1000015          5       1   228        78          3.0           0.34   
3  1000019          3       0    57       166          2.0           0.00   
4  1000023          2       1   293       164          2.0           0.34   

   romCapacity  romLeftRation  color  fontSize  ct  carrier  os  bootTimes  \
0         32.0           0.46     80      1.15   5        1  14        108   
1          0.0           0.00     16      0.00   6        1  15          0   
2         32.0           0.06     80      1.30   8        2  14         12   
3         17.0           0.00    100      0.00   7        3  15          0   
4         16.0           0.06    119      1.00   8        2  12          5   

   AFuncTimes  BFuncTimes  CFuncTimes  DFuncTimes  EFuncTimes  FFunc

In [52]:
gc.collect()

40

In [51]:
final_df.to_csv("2nd_feature_engineering_2010000_114.csv",index=0)

In [11]:
df_train = pd.read_csv("2nd_feature_engineering_2010000_114.csv")

print(df_train.head())

print(df_train.shape)

       uid  age_group  gender  city  prodName  ramCapacity  ramLeftRation  \
0  1000001          4       0    51        78          3.0           0.43   
1  1000011          3       0    30       138          0.0           0.00   
2  1000015          5       1   228        78          3.0           0.34   
3  1000019          3       0    57       166          2.0           0.00   
4  1000023          2       1   293       164          2.0           0.34   

   romCapacity  romLeftRation  color  fontSize  ct  carrier  os  bootTimes  \
0         32.0           0.46     80      1.15   5        1  14        108   
1          0.0           0.00     16      0.00   6        1  15          0   
2         32.0           0.06     80      1.30   8        2  14         12   
3         17.0           0.00    100      0.00   7        3  15          0   
4         16.0           0.06    119      1.00   8        2  12          5   

   AFuncTimes  BFuncTimes  CFuncTimes  DFuncTimes  EFuncTimes  FFunc

In [12]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
#df_test = mergeAppDataUpgrade(mergeBasicTables(age_test))
#df_test.to_csv("add_40_actived_classes_features_for_testSet.csv",index=0)
#final_test_df = pd.merge(df_test, tempdf, how='left', on='uid').fillna(0)
#final_test_df.to_csv('2nd_feature_engineering_testSet_502500_113.csv',index=0)


In [16]:
df_test = pd.read_csv('2nd_feature_engineering_testSet_502500_113.csv')
gc.collect()

0

In [62]:
print(df_train)
print(df_test)

             uid  age_group  gender  city  prodName  ramCapacity  \
0        1000001          4       0    51        78          3.0   
1        1000011          3       0    30       138          0.0   
2        1000015          5       1   228        78          3.0   
3        1000019          3       0    57       166          2.0   
4        1000023          2       1   293       164          2.0   
5        1000025          4       0   318        81          4.0   
6        1000029          4       0   355        64          6.0   
7        1000035          2       0   239        64          6.0   
8        1000037          4       0   248        10          4.0   
9        1000038          3       0   336        13          4.0   
10       1000040          3       0   252        49          6.0   
11       1000044          4       0   330        66          6.0   
12       1000046          3       0   135       165          4.0   
13       1000051          3       1    30       

[2010000 rows x 114 columns]
            uid  gender  city  prodName  ramCapacity  ramLeftRation  \
0       1000002       1   298       113          0.0           0.26   
1       1000003       0   222         6          8.0           0.35   
2       1000004       1   198        31          3.0           0.36   
3       1000005       0   107         1          6.0           0.00   
4       1000006       0   284       146          2.0           0.19   
5       1000007       1   314       191          6.0           0.20   
6       1000008       1   298        52          6.0           0.36   
7       1000009       0   336        52          6.0           0.39   
8       1000010       0   298       128          6.0           0.36   
9       1000012       0   201        66          4.0           0.31   
10      1000013       0   133        50          4.0           0.49   
11      1000014       1   314         3          6.0           0.49   
12      1000016       0   348       130         

[502500 rows x 113 columns]


In [26]:
print("开始训练模型：")
param = {
        'learning_rate': 0.13,
        'lambda_l1': 1,
        'lambda_l2': 0.01,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': {'multi_error'}
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=2000, valid_sets=[vali], early_stopping_rounds=50)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

开始训练模型：
0
training start...
[1]	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.676951
[3]	valid_0's multi_error: 0.653935
[4]	valid_0's multi_error: 0.634784
[5]	valid_0's multi_error: 0.612902
[6]	valid_0's multi_error: 0.59788
[7]	valid_0's multi_error: 0.588113
[8]	valid_0's multi_error: 0.580389
[9]	valid_0's multi_error: 0.575157
[10]	valid_0's multi_error: 0.570362
[11]	valid_0's multi_error: 0.566151
[12]	valid_0's multi_error: 0.562414
[13]	valid_0's multi_error: 0.559547
[14]	valid_0's multi_error: 0.556666
[15]	valid_0's multi_error: 0.554274
[16]	valid_0's multi_error: 0.552428
[17]	valid_0's multi_error: 0.550374
[18]	valid_0's multi_error: 0.548936
[19]	valid_0's multi_error: 0.547435
[20]	valid_0's multi_error: 0.545902
[21]	valid_0's multi_error: 0.54438
[22]	valid_0's multi_error: 0.542971
[23]	valid_0's multi_error: 0.541547
[24]	valid_0's multi_error: 0.539942
[25]	valid_0's multi_error: 0.538

[432]	valid_0's multi_error: 0.47478
[433]	valid_0's multi_error: 0.474743
[434]	valid_0's multi_error: 0.474772
[435]	valid_0's multi_error: 0.474769
[436]	valid_0's multi_error: 0.474759
[437]	valid_0's multi_error: 0.474762
[438]	valid_0's multi_error: 0.474753
[439]	valid_0's multi_error: 0.474732
[440]	valid_0's multi_error: 0.474747
[441]	valid_0's multi_error: 0.47476
[442]	valid_0's multi_error: 0.474789
[443]	valid_0's multi_error: 0.474775
[444]	valid_0's multi_error: 0.474752
[445]	valid_0's multi_error: 0.47475
[446]	valid_0's multi_error: 0.474784
[447]	valid_0's multi_error: 0.474725
[448]	valid_0's multi_error: 0.474687
[449]	valid_0's multi_error: 0.474658
[450]	valid_0's multi_error: 0.474566
[451]	valid_0's multi_error: 0.474555
[452]	valid_0's multi_error: 0.474547
[453]	valid_0's multi_error: 0.474575
[454]	valid_0's multi_error: 0.474566
[455]	valid_0's multi_error: 0.474526
[456]	valid_0's multi_error: 0.47455
[457]	valid_0's multi_error: 0.474549
[458]	valid_0's 

[864]	valid_0's multi_error: 0.470914
[865]	valid_0's multi_error: 0.470892
[866]	valid_0's multi_error: 0.470896
[867]	valid_0's multi_error: 0.470901
[868]	valid_0's multi_error: 0.470855
[869]	valid_0's multi_error: 0.470866
[870]	valid_0's multi_error: 0.470884
[871]	valid_0's multi_error: 0.470889
[872]	valid_0's multi_error: 0.470878
[873]	valid_0's multi_error: 0.470893
[874]	valid_0's multi_error: 0.470886
[875]	valid_0's multi_error: 0.47088
[876]	valid_0's multi_error: 0.470863
[877]	valid_0's multi_error: 0.470852
[878]	valid_0's multi_error: 0.470834
[879]	valid_0's multi_error: 0.470843
[880]	valid_0's multi_error: 0.470868
[881]	valid_0's multi_error: 0.47089
[882]	valid_0's multi_error: 0.470892
[883]	valid_0's multi_error: 0.470874
[884]	valid_0's multi_error: 0.470884
[885]	valid_0's multi_error: 0.470893
[886]	valid_0's multi_error: 0.470902
[887]	valid_0's multi_error: 0.470886
[888]	valid_0's multi_error: 0.470883
[889]	valid_0's multi_error: 0.470832
[890]	valid_0'

[1288]	valid_0's multi_error: 0.469323
[1289]	valid_0's multi_error: 0.469344
[1290]	valid_0's multi_error: 0.469398
[1291]	valid_0's multi_error: 0.469404
[1292]	valid_0's multi_error: 0.469386
[1293]	valid_0's multi_error: 0.469378
[1294]	valid_0's multi_error: 0.46931
[1295]	valid_0's multi_error: 0.469301
[1296]	valid_0's multi_error: 0.469307
[1297]	valid_0's multi_error: 0.469295
[1298]	valid_0's multi_error: 0.46934
[1299]	valid_0's multi_error: 0.469344
[1300]	valid_0's multi_error: 0.469314
[1301]	valid_0's multi_error: 0.469338
[1302]	valid_0's multi_error: 0.469347
[1303]	valid_0's multi_error: 0.469347
[1304]	valid_0's multi_error: 0.46936
[1305]	valid_0's multi_error: 0.469396
[1306]	valid_0's multi_error: 0.469377
[1307]	valid_0's multi_error: 0.469365
[1308]	valid_0's multi_error: 0.469356
[1309]	valid_0's multi_error: 0.469355
[1310]	valid_0's multi_error: 0.469334
[1311]	valid_0's multi_error: 0.469317
[1312]	valid_0's multi_error: 0.469329
[1313]	valid_0's multi_error

[399]	valid_0's multi_error: 0.476178
[400]	valid_0's multi_error: 0.476134
[401]	valid_0's multi_error: 0.476118
[402]	valid_0's multi_error: 0.476082
[403]	valid_0's multi_error: 0.476057
[404]	valid_0's multi_error: 0.476042
[405]	valid_0's multi_error: 0.476012
[406]	valid_0's multi_error: 0.476
[407]	valid_0's multi_error: 0.475999
[408]	valid_0's multi_error: 0.475909
[409]	valid_0's multi_error: 0.475931
[410]	valid_0's multi_error: 0.475918
[411]	valid_0's multi_error: 0.475875
[412]	valid_0's multi_error: 0.475775
[413]	valid_0's multi_error: 0.475796
[414]	valid_0's multi_error: 0.475724
[415]	valid_0's multi_error: 0.47574
[416]	valid_0's multi_error: 0.475746
[417]	valid_0's multi_error: 0.475685
[418]	valid_0's multi_error: 0.475693
[419]	valid_0's multi_error: 0.475679
[420]	valid_0's multi_error: 0.475651
[421]	valid_0's multi_error: 0.475627
[422]	valid_0's multi_error: 0.475618
[423]	valid_0's multi_error: 0.475585
[424]	valid_0's multi_error: 0.475554
[425]	valid_0's 

[831]	valid_0's multi_error: 0.472116
[832]	valid_0's multi_error: 0.472063
[833]	valid_0's multi_error: 0.472072
[834]	valid_0's multi_error: 0.472061
[835]	valid_0's multi_error: 0.472052
[836]	valid_0's multi_error: 0.47203
[837]	valid_0's multi_error: 0.472031
[838]	valid_0's multi_error: 0.472018
[839]	valid_0's multi_error: 0.471976
[840]	valid_0's multi_error: 0.471988
[841]	valid_0's multi_error: 0.472004
[842]	valid_0's multi_error: 0.472028
[843]	valid_0's multi_error: 0.471997
[844]	valid_0's multi_error: 0.471997
[845]	valid_0's multi_error: 0.471966
[846]	valid_0's multi_error: 0.47196
[847]	valid_0's multi_error: 0.471927
[848]	valid_0's multi_error: 0.471949
[849]	valid_0's multi_error: 0.471939
[850]	valid_0's multi_error: 0.47191
[851]	valid_0's multi_error: 0.471915
[852]	valid_0's multi_error: 0.471916
[853]	valid_0's multi_error: 0.471934
[854]	valid_0's multi_error: 0.471916
[855]	valid_0's multi_error: 0.471885
[856]	valid_0's multi_error: 0.4719
[857]	valid_0's m

[1256]	valid_0's multi_error: 0.470372
[1257]	valid_0's multi_error: 0.470346
[1258]	valid_0's multi_error: 0.470336
[1259]	valid_0's multi_error: 0.470351
[1260]	valid_0's multi_error: 0.47033
[1261]	valid_0's multi_error: 0.470306
[1262]	valid_0's multi_error: 0.470325
[1263]	valid_0's multi_error: 0.470349
[1264]	valid_0's multi_error: 0.470349
[1265]	valid_0's multi_error: 0.470375
[1266]	valid_0's multi_error: 0.470419
[1267]	valid_0's multi_error: 0.470419
[1268]	valid_0's multi_error: 0.470387
[1269]	valid_0's multi_error: 0.470416
[1270]	valid_0's multi_error: 0.470413
[1271]	valid_0's multi_error: 0.470396
[1272]	valid_0's multi_error: 0.470373
[1273]	valid_0's multi_error: 0.470354
[1274]	valid_0's multi_error: 0.470376
[1275]	valid_0's multi_error: 0.470369
[1276]	valid_0's multi_error: 0.470339
[1277]	valid_0's multi_error: 0.470327
[1278]	valid_0's multi_error: 0.470307
[1279]	valid_0's multi_error: 0.470319
[1280]	valid_0's multi_error: 0.47031
[1281]	valid_0's multi_erro

[336]	valid_0's multi_error: 0.478313
[337]	valid_0's multi_error: 0.478262
[338]	valid_0's multi_error: 0.478207
[339]	valid_0's multi_error: 0.478129
[340]	valid_0's multi_error: 0.478057
[341]	valid_0's multi_error: 0.477999
[342]	valid_0's multi_error: 0.477944
[343]	valid_0's multi_error: 0.477968
[344]	valid_0's multi_error: 0.477975
[345]	valid_0's multi_error: 0.477947
[346]	valid_0's multi_error: 0.477878
[347]	valid_0's multi_error: 0.477844
[348]	valid_0's multi_error: 0.477814
[349]	valid_0's multi_error: 0.477795
[350]	valid_0's multi_error: 0.477735
[351]	valid_0's multi_error: 0.477714
[352]	valid_0's multi_error: 0.477644
[353]	valid_0's multi_error: 0.477617
[354]	valid_0's multi_error: 0.47759
[355]	valid_0's multi_error: 0.477611
[356]	valid_0's multi_error: 0.477656
[357]	valid_0's multi_error: 0.477601
[358]	valid_0's multi_error: 0.477595
[359]	valid_0's multi_error: 0.477571
[360]	valid_0's multi_error: 0.477516
[361]	valid_0's multi_error: 0.477474
[362]	valid_0

[768]	valid_0's multi_error: 0.472689
[769]	valid_0's multi_error: 0.472629
[770]	valid_0's multi_error: 0.472651
[771]	valid_0's multi_error: 0.472689
[772]	valid_0's multi_error: 0.472716
[773]	valid_0's multi_error: 0.47271
[774]	valid_0's multi_error: 0.472686
[775]	valid_0's multi_error: 0.472711
[776]	valid_0's multi_error: 0.472707
[777]	valid_0's multi_error: 0.472711
[778]	valid_0's multi_error: 0.472696
[779]	valid_0's multi_error: 0.472719
[780]	valid_0's multi_error: 0.472735
[781]	valid_0's multi_error: 0.472705
[782]	valid_0's multi_error: 0.472662
[783]	valid_0's multi_error: 0.472659
[784]	valid_0's multi_error: 0.472668
[785]	valid_0's multi_error: 0.472666
[786]	valid_0's multi_error: 0.472662
[787]	valid_0's multi_error: 0.472651
[788]	valid_0's multi_error: 0.472645
[789]	valid_0's multi_error: 0.472641
[790]	valid_0's multi_error: 0.472651
[791]	valid_0's multi_error: 0.472666
[792]	valid_0's multi_error: 0.472641
[793]	valid_0's multi_error: 0.472637
[794]	valid_0

In [27]:
predicts = model.predict(x_vali)
predicts = [np.argmax(x) for x in predicts]
cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

results = []
for line in cv_pred:
    results.append(np.argmax(np.bincount(line)))

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
acc_score = accuracy_score(y_vali, predicts)


print("accuracy_score is %f "% acc_score)

accuracy_score is 0.528731 


In [28]:
#保存  每次超参数组合以及测试集上得到的acc  到文件中

path_csv='Record_param_acc_lost'+'.csv'
for (k,v) in  param.items():
    SaveFile2csv(path_csv, [k, str(v)])
    
#record_parameters_auc_loss_txt.append('\nlost:'+answer_detail+\
#                '\n-------------------------------------------------------------------------------\n')
#print(record_parameters_auc_loss_csv)
SaveFile2csv(path_csv, ['\nAccuracy score', str(acc_score)])